# Circular References

Object A(var_1) -> <- Object B(var 2)

These objects reference each other in a loop and they are unable to be garbage collected and `Memory leak` happens.


# Garbage Collector

- can be controlled using the `gc` module
- by default it is turned on
- can be turned off but I must make sure my code does not create circular references
- runs periodically on its own
- can be called manually, and even do my own cleanup
- `gc` module sometimes does not work properly for Python < 3.4

In [1]:
import ctypes
import gc

In [5]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [6]:
# Looks for all the objects that are tracked by the garbage collector and will try to find the object I need by comparing the ids
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"

In [9]:
# setting up two classes with the circular reference
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b: {1}'.format(hex(id(self)), hex(id(self.b))))
        
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

In [11]:
# garbage collector disabled
gc.disable()

In [14]:
# memory address for a in both classes is the same and for b in both classes is the same
my_var = A()

B: self: 0x1154c8438, a: 0x1154c8860
A: self: 0x1154c8860, b: 0x1154c8438


In [16]:
# my_var points to a
hex(id(my_var))

'0x1154c8860'

In [21]:
# storing memory addresses
a_id = id(my_var)
b_id = id(my_var.b)

In [22]:
print(hex(a_id), hex(b_id))

0x1154c8860 0x1154c8438


In [23]:
ref_count(a_id)

2

In [24]:
ref_count(b_id)

1

In [25]:
object_by_id(a_id)

'Object exists'

In [27]:
object_by_id(b_id)

'Object exists'

In [28]:
# destroying reference between my_var and A object
my_var = None

In [35]:
# getting rid of my_var, but there are still pointers between A and B - circular reference
ref_count(a_id)

1

In [36]:
ref_count(b_id)

1

In [39]:
# they still exist because I disabled garbage collector
object_by_id(a_id)

'Object exists'

In [40]:
object_by_id(b_id)

'Object exists'

In [41]:
# running garbage collector manually
gc.collect()

1957

In [42]:
object_by_id(a_id)

'Not found'

In [43]:
object_by_id(b_id)

'Not found'

In [44]:
ref_count(a_id)

0

In [45]:
ref_count(b_id)

0